In [3]:
import cv2

cap = cv2.VideoCapture('ExampleBGSubtraction.avi')
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)

rate = int(1000/fps)

while(cap.isOpened()):
    haveFrame, im = cap.read()

    if (not haveFrame) or (cv2.waitKey(rate) & 0xFF == ord('q')):
        break

    cv2.imshow('video',im)

cap.release()
cv2.destroyAllWindows()

25.0


In [4]:
cap = cv2.VideoCapture('ExampleBGSubtraction.avi')
haveFrame,bg = cap.read() # get bg from first frame

while(cap.isOpened()):
    haveFrame,im = cap.read()

    if (not haveFrame) or (cv2.waitKey(1) & 0xFF == ord('q')):
        break

    diffc = cv2.absdiff(im,bg)
    diffg = cv2.cvtColor(diffc,cv2.COLOR_BGR2GRAY)
    bwmask = cv2.inRange(diffg,50,255)

    cv2.imshow('diffc', diffc)
    cv2.moveWindow('diffc',10,10)
    cv2.imshow('diffg',diffg)
    cv2.moveWindow('diffg', 400, 10)
    cv2.imshow('bwmask', bwmask)
    cv2.moveWindow('bwmask', 800, 10)

cap.release()
cv2.destroyAllWindows()

In [8]:
cap = cv2.VideoCapture('ExampleBGSubtraction.avi')
haveFrame,bg = cap.read()

while(cap.isOpened()):
    haveFrame,im = cap.read()

    if (not haveFrame) or (cv2.waitKey(1) & 0xFF == ord('q')):
        break

    diffc = cv2.absdiff(im,bg)
    diffg = cv2.cvtColor(diffc,cv2.COLOR_BGR2GRAY)
    bwmask = cv2.inRange(diffg,50,255)

    bwmask_median = cv2.medianBlur(bwmask, 5)

    contours,hierarchy = cv2.findContours(bwmask_median, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE) #for opencv 4.x.x
    #contourmask,contours,hierarchy = cv2.findContours(bwmask_median, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE) #for opencv 3.2.x 3.4.x
    #contourmask,contours,hierarchy = cv2.findContours(bwmask_median.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE) #for opencv 3.1.x

    im_out_contour = im.copy()
    cv2.drawContours(im_out_contour, contours, -1, (0, 255, 0), 1)

    im_out_boundingbox = im.copy()
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(im_out_boundingbox, (x, y), (x + w, y + h), (0, 0, 255), 2)


    cv2.imshow('bwmask', bwmask)
    cv2.moveWindow('bwmask',10,10)
    cv2.imshow('bwmask_median', bwmask_median)
    cv2.moveWindow('bwmask_median', 400, 10)
    cv2.imshow('im_out_contour', im_out_contour)
    cv2.moveWindow('im_out_contour', 10, 350)
    cv2.imshow('im_out_boundingbox', im_out_boundingbox)
    cv2.moveWindow('im_out_boundingbox', 400, 350)
    
    #cv2.waitKey()

cap.release()
cv2.destroyAllWindows()

In [9]:
import numpy as np

im = np.zeros(shape =(400,400)).astype('uint8')
polygons = [np.array([[20,20],[120,50],[30,80]]),
            np.array([[200,200],[200,350],[350,350], [350,200] ]),
            np.array([[170, 350], [60, 170], [130, 140]])]
colors = [255,255,255]
cv2.fillPoly(im,polygons,colors)
im[250:340,250:340] = 0
im[270:300,290:320] = 255

contours,hierarchy = cv2.findContours(im, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
#try mode cv2.RETR_EXTERNAL and cv2.RETR_TREE
#try method cv2.CHAIN_APPROX_NONE and cv2.CHAIN_APPROX_SIMPLE


print('Types of contours:', type(contours))
print('Length of contours:', len(contours))

n = 0
print('Types of each contour:', type(contours[n]))
print('Shape of each contour:', contours[n].shape)
print('Each contour value (list of points):\n',contours[n])

out = np.zeros(shape =(400,400)).astype('uint8')
for pos in contours[n]:
    out[pos[0,1],pos[0,0]] = 255

#print('Hierarchy:\n', 'Left Right Child Parent\n', hierarchy)

cv2.imshow('im', im)
cv2.imshow('out', out)

cv2.waitKey()

Types of contours: <class 'tuple'>
Length of contours: 5
Types of each contour: <class 'numpy.ndarray'>
Shape of each contour: (600, 1, 2)
Each contour value (list of points):
 [[[200 200]]

 [[200 201]]

 [[200 202]]

 ...

 [[203 200]]

 [[202 200]]

 [[201 200]]]


113

In [1]:
import numpy as np
import cv2

cap = cv2.VideoCapture('ExampleBGSubtraction.avi')

haveFrame,bg = cap.read()

while(cap.isOpened()):
    haveFrame,im = cap.read()

    if (not haveFrame) or (cv2.waitKey(100) & 0xFF == ord('q')):
        break

    diffc = cv2.absdiff(im,bg)
    diffg = cv2.cvtColor(diffc,cv2.COLOR_BGR2GRAY)
    bwmask = cv2.inRange(diffg,50,255)

    bwmask_median = cv2.medianBlur(bwmask,5)

    kernel = np.ones((55,25), np.uint8)
    bwmask_close = cv2.morphologyEx(bwmask_median, cv2.MORPH_CLOSE, kernel)

    contours,hierarchy = cv2.findContours(bwmask_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    im_out_contour = im.copy()
    cv2.drawContours(im_out_contour, contours, -1, (0, 255, 0), 1)

    im_out_boundingbox = im.copy()
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(im_out_boundingbox, (x, y), (x + w, y + h), (0, 0, 255), 2)


    cv2.imshow('bwmask', bwmask)
    cv2.moveWindow('bwmask',10,10)
    cv2.imshow('bwmask_median', bwmask_median)
    cv2.moveWindow('bwmask_median', 380, 10)
    cv2.imshow('bwmask_close', bwmask_close)
    cv2.moveWindow('bwmask_close', 750, 10)
    cv2.imshow('im_out_contour', im_out_contour)
    cv2.moveWindow('im_out_contour', 10, 350)
    cv2.imshow('im_out_boundingbox', im_out_boundingbox)
    cv2.moveWindow('im_out_boundingbox', 380, 350)

    #cv2.waitKey()

cap.release()
cv2.destroyAllWindows()